# Backpropagation Practice

Implement a 3 input, 4 node hidden-layer, 1 output node Multilayer Perceptron on the following dataset:

| x1 | x2 | x3 | y |
|----|----|----|---|
| 0  | 0  | 1  | 0 |
| 0  | 1  | 1  | 1 |
| 1  | 0  | 1  | 1 |
| 0  | 1  | 0  | 1 |
| 1  | 0  | 0  | 1 |
| 1  | 1  | 1  | 0 |
| 0  | 0  | 0  | 0 |

If you look at the data you'll notice that the first two columns behave like an XOR gate while the last column is mostly just noise. Remember that creating an XOR gate was what the perceptron was criticized for not being able to learn. 

In [276]:
import numpy as np
X = np.array([[0,0,1], [0,1,1], [1,0,1], [0,1,0], [1,0,0], [1,1,1], [0,0,0]])
y = np.array([0,1,1,1,1,0,0]).reshape(-1,1)

In [277]:
X.shape, y.shape

((7, 3), (7, 1))

In [290]:
class NeuralNetwork: 
    def __init__(self, inputs, hiddenNodes, outputNodes):
        # Set upArchietecture 
        self.inputs = inputs
        self.hiddenNodes = hiddenNodes
        self.outputNodes = outputNodes
        
        #Initial weights
        self.weights1 = np.random.randn(self.inputs, self.hiddenNodes) 
        self.weights2 = np.random.rand(self.hiddenNodes, self.outputNodes) 
    
    def sigmoid(self, s):
        return 1 / (1+np.exp(-s))
    
    def sigmoidPrime(self, s):
        #derivative of the sigmoid
        return s * (1 - s)
    
    def feed_forward(self, X):
        """
        Calculate the NN inference using feed forward.
        """
        
        #Weighted sume of inputs and hidden layer
        self.hidden_sum = np.dot(X, self.weights1)
        
        #Acivations of weighted sum
        self.activated_hidden = self.sigmoid(self.hidden_sum)
        
        # Weight sum between hidden and output
        self.output_sum = np.dot(self.activated_hidden, self.weights2)
        
        #Final activation of output
        self.activated_output = self.sigmoid(self.output_sum)
        return self.activated_output
    
    def backward(self, X, y, o):
        """
        Backward propagate through the network
        """
        self.o_error = y - o #error in output
        self.o_delta = self.o_error * self.sigmoidPrime(o) # apply derivative of sigmoid to error
        
        self.z2_error = self.o_delta.dot(self.weights2.T) # z2 error: how much our hidden layer weights were off
        self.z2_delta = self.z2_error*self.sigmoidPrime(self.activated_hidden)
        
        self.weights1 += X.T.dot(self.z2_delta) #Adjust first set (input => hidden) weights
        self.weights2 += self.activated_hidden.T.dot(self.o_delta) #adjust second set (hidden => output) weights
        
    def train(self, X, y):
        o = self.feed_forward(X)
        self.backward(X, y, o)
            
    def get_attributes(self):
        attributes = ['weights1', 'hidden_sum', 'activated_hidden', 'weights2', 'output']
        [print(i + '\n', getattr(nn,i), '\n'+'---'*3) for i in dir(nn) if i in attributes]

In [291]:
nn = NeuralNetwork(inputs=3, hiddenNodes=4, outputNodes=1)

for i in range(10000):
    if ((i+1) % 5000 == 0):
        print('+' + '---' * 3 + f'EPOCH {i+1}' + '---'*3 + '+')
        print('Input: \n', X)
        print('Actual Output: \n', y)
        print('Predicted Output: \n', str(nn.feed_forward(X)))
        print("Loss: \n", str(np.mean(np.square(y - nn.feed_forward(X)))))
    nn.train(X,y)

+---------EPOCH 5000---------+
Input: 
 [[0 0 1]
 [0 1 1]
 [1 0 1]
 [0 1 0]
 [1 0 0]
 [1 1 1]
 [0 0 0]]
Actual Output: 
 [[0]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]]
Predicted Output: 
 [[0.00420612]
 [0.98186607]
 [0.98807408]
 [0.98340245]
 [0.98957603]
 [0.02049924]
 [0.02690924]]
Loss: 
 0.00028817460986999536
+---------EPOCH 10000---------+
Input: 
 [[0 0 1]
 [0 1 1]
 [1 0 1]
 [0 1 0]
 [1 0 0]
 [1 1 1]
 [0 0 0]]
Actual Output: 
 [[0]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]]
Predicted Output: 
 [[0.00280092]
 [0.98756335]
 [0.99141769]
 [0.98868473]
 [0.99257264]
 [0.01422455]
 [0.01838031]]
Loss: 
 0.00013707802034756972


## Try building/training a more complex MLP on a bigger dataset.

Use the [MNIST dataset](http://yann.lecun.com/exdb/mnist/) to build the cannonical handwriting digit recognizer and see what kind of accuracy you can achieve. 

If you need inspiration, the internet is chalk-full of tutorials, but I want you to see how far you can get on your own first. I've linked to the original MNIST dataset above but it will probably be easier to download data through a neural network library. If you reference outside resources make sure you understand every line of code that you're using from other sources, and share with your fellow students helpful resources that you find.


In [250]:
import idx2numpy

train_file = 'data/train-images.idx3-ubyte'
train_raw = idx2numpy.convert_from_file(train_file)

train_label_file = 'data/train-labels.idx1-ubyte'
train_labels = idx2numpy.convert_from_file(train_label_file).reshape(-1,1)

train_raw.shape, train_labels.shape

((60000, 28, 28), (60000, 1))

In [251]:
train_flat = []
for i in range(len(train)):
    train_flat.append(train[i].flatten())
    
train_flat = np.array(train_flat)
train_flat.shape

(60000, 784)

In [253]:
import keras

X = train_flat.astype('float32')
y = train_labels.astype('float32')
y = keras.utils.to_categorical(y, 10)

Using TensorFlow backend.


In [254]:
y[0]

array([0., 0., 0., 0., 0., 1., 0., 0., 0., 0.], dtype=float32)

In [269]:
nn = NeuralNetwork(inputs=784, hiddenNodes=28, outputNodes=10)

In [270]:
#batches, test on first 500:

train_counter = 0
X_batch = X[train_counter:train_counter+500]
y_batch = y[train_counter:train_counter+500]

In [271]:
%%time
for i in range(1000):
    nn.train(X_batch,y_batch)

/home/jm/.local/lib/python3.6/site-packages/ipykernel_launcher.py:13: RuntimeWarning: overflow encountered in exp
  del sys.path[0]


CPU times: user 24.4 s, sys: 11.6 s, total: 36 s
Wall time: 10.6 s


In [272]:
nn.activated_output.shape

(500, 10)

In [273]:
import pandas as pd
pd.DataFrame(nn.activated_output)

,0,1,2,3,4,5,6,7,8,9
0,1.454558e-05,1.142568e-70,1.617251e-41,6.569474e-08,5.900727e-70,7.482440e-22,9.282407e-06,1.040981e-41,7.571148e-07,3.363487e-10
1,9.075705e-07,1.241320e-102,1.936808e-48,8.168581e-10,1.312897e-92,5.915005e-30,5.814812e-08,1.027816e-57,1.315416e-09,2.422488e-13
2,6.312361e-06,7.889795e-99,6.156182e-45,2.599255e-09,1.427702e-82,6.545920e-25,4.770888e-06,6.426665e-53,7.197100e-11,4.456480e-11
3,2.485669e-08,1.837398e-138,3.113941e-61,1.092224e-12,1.883683e-119,8.547243e-37,6.103824e-08,1.142076e-75,1.431872e-12,3.725533e-15
4,3.358239e-07,4.414933e-117,1.803737e-58,7.949093e-12,1.170859e-105,2.767751e-33,1.728271e-07,1.048390e-64,4.360828e-11,8.087662e-14
5,3.155080e-08,3.162952e-127,8.106345e-59,1.654973e-12,5.669653e-112,8.370062e-35,8.084930e-08,6.487634e-71,5.175120e-12,2.945921e-14
6,1.150189e-07,2.866167e-113,4.973746e-55,1.888403e-11,3.828937e-99,2.603401e-32,8.698254e-08,5.876136e-64,1.472485e-11,2.116876e-14
7,1.333112e-07,1.406613e-112,1.766055e-56,1.112138e-11,2.616130e-102,1.520014e-32,1.479542e-07,1.671930e-63,1.664514e-10,1.394019e-13
8,8.495874e-07,1.695550e-102,2.985576e-51,1.981958e-10,2.335726e-94,7.975016e-31,1.053047e-07,9.997706e-60,1.405465e-10,3.798221e-14
9,1.234012e-07,2.980944e-119,5.998671e-54,3.721557e-12,7.685199e-105,6.231430e-32,7.583108e-07,5.955647e-65,3.333972e-11,5.965189e-14


In [ ]:
#no idea how to get the NN to output 0-9 

## Stretch Goals: 

- Implement Cross Validation model evaluation on your MNIST implementation 
- Research different [Gradient Descent Based Optimizers](https://keras.io/optimizers/)
 - [Siraj Raval the evolution of gradient descent](https://www.youtube.com/watch?v=nhqo0u1a6fw)
- Build a housing price estimation model using a neural network. How does its accuracy compare with the regression models that we fit earlier on in class?